Here I have used a the file scraped on 31 Dec 2017 to continue building my library of positive and negative tweets. The aim of this exercise is to improve the accuracy of my original Naive Bayes model.

In [2]:
import numpy as np
import pandas as pd
import re
import nltk.classify.util
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.classify import NaiveBayesClassifier

%matplotlib inline


In [3]:
df = pd.read_csv('game_reviews_31_12_17.csv', encoding = 'latin1').unstack()

In [12]:
df2 = df[80:]

In [14]:
df2[:15]

0  0     (' 2 ', '  While the gameplay is fun, the game...
   1     (' 5 ', '  Its a very enjoyable game. What a p...
   2     (' 5 ', "  Very fun game, it's worth the money...
   3     (' 4 ', '  It\'s a nice short game, worth the ...
   4     (' 3 ', '  Its a great app but the only issue ...
   5                                   (' 5 ', '  Cool  ')
   6     (' 1 ', "  It does nothing Let's Just say  IF ...
   7     (' 5 ', '  Awesome Game.  I love the interface...
   8     (' 5 ', "  Wow, rarely give all 5 stars! I nee...
   9     (' 5 ', "  I dislike the app's icon.. Pls chan...
   10    (' 5 ', '  I think there is a bug . When I cli...
   11    (' 3 ', "  Fortunately I got this game free du...
   12          (' 5 ', '  Good  gameplay  graphics bor  ')
   13    (' 5 ', "  Great game! Challenging, and logica...
   14    (' 5 ', "  Fun and tough. A puzzle game that's...
dtype: object

In [22]:
df2[2][3]

str

In [16]:
df2[2][11:]

'Very fun game, it\'s worth the money. But there are a couple problems. 1. Way too short I managed to beat the game within 2 hours.  2. The secrets are pretty easy, didn\'t need to repeat any levels at all. unless there are more then 3 secrets, then I didn\'t find them all. A good thing about the game on the other hand is that every single level you can play slow and steady or just rush through the level really quickly, I  like that.  ")'

In [28]:
stars = []
comments = []
for i in np.arange(0, len(df2)):
    try:
        star = df2[i][3]
        stars.append(star)
        comment = df2[i][11:]
        comments.append(comment)
    except:
        pass
    
print(stars, comments)

['2', '5', '5', '4', '3', '5', '1', '5', '5', '5', '5', '3', '5', '5', '5', '3', '5', '3', '5', '5', '5', '5', '1', '3', '5', '5', '5', '5', '5', '3', '3', '5', '5', '5', '5', '2', '5', '5', '4', '3', '5', '1', '5', '5', '5', '5', '3', '5', '5', '5', '3', '5', '3', '5', '5', '5', '5', '1', '3', '5', '5', '5', '5', '5', '3', '3', '5', '5', '5', '5', '5', '4', '5', '2', '1', '2', '5', '5', '5', '5', '3', '2', '4', '5', '5', '5', '2', '3', '5', '4', '5', '1', '3', '1', '5', '5', '5', '5', '4', '4', '3', '5', '4', '5', '2', '1', '2', '5', '5', '5', '5', '3', '2', '4', '5', '5', '5', '2', '3', '5', '4', '5', '1', '3', '1', '5', '5', '5', '5', '4', '4', '3', '3', '4', '5', '4', '3', '3', '5', '5', '3', '4', '1', '4', '5', '3', '5', '1', '4', '5', '4', '5', '5', '5', '5', '5', '5', '2', '3', '4', '5', '4', '3', '3', '5', '5', '3', '4', '1', '4', '5', '3', '5', '1', '4', '5', '4', '5', '5', '5', '5', '5', '5', '2', '1', '2', '5', '3', '5', '4', '4', '1', '5', '4', '1', '3', '5', '1', '3', '4',

/Users/cherylto/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/multi.py:819: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return self._engine.get_value(s, k)


In [27]:
len(stars)

312

In [29]:
len(comments)

312

In [30]:
stars_numeric = pd.to_numeric(stars)
stars_numeric

array([2, 5, 5, 4, 3, 5, 1, 5, 5, 5, 5, 3, 5, 5, 5, 3, 5, 3, 5, 5, 5, 5, 1,
       3, 5, 5, 5, 5, 5, 3, 3, 5, 5, 5, 5, 2, 5, 5, 4, 3, 5, 1, 5, 5, 5, 5,
       3, 5, 5, 5, 3, 5, 3, 5, 5, 5, 5, 1, 3, 5, 5, 5, 5, 5, 3, 3, 5, 5, 5,
       5, 5, 4, 5, 2, 1, 2, 5, 5, 5, 5, 3, 2, 4, 5, 5, 5, 2, 3, 5, 4, 5, 1,
       3, 1, 5, 5, 5, 5, 4, 4, 3, 5, 4, 5, 2, 1, 2, 5, 5, 5, 5, 3, 2, 4, 5,
       5, 5, 2, 3, 5, 4, 5, 1, 3, 1, 5, 5, 5, 5, 4, 4, 3, 3, 4, 5, 4, 3, 3,
       5, 5, 3, 4, 1, 4, 5, 3, 5, 1, 4, 5, 4, 5, 5, 5, 5, 5, 5, 2, 3, 4, 5,
       4, 3, 3, 5, 5, 3, 4, 1, 4, 5, 3, 5, 1, 4, 5, 4, 5, 5, 5, 5, 5, 5, 2,
       1, 2, 5, 3, 5, 4, 4, 1, 5, 4, 1, 3, 5, 1, 3, 4, 5, 5, 5, 5, 5, 5, 4,
       1, 2, 5, 3, 5, 4, 4, 1, 5, 4, 1, 3, 5, 1, 3, 4, 5, 5, 5, 5, 5, 5, 4,
       3, 5, 5, 4, 5, 5, 5, 5, 2, 1, 1, 4, 5, 4, 5, 3, 5, 5, 5, 4, 1, 3, 5,
       5, 4, 5, 5, 5, 5, 2, 1, 1, 4, 5, 4, 5, 3, 5, 5, 5, 4, 1, 5, 5, 3, 3,
       1, 5, 4, 4, 3, 2, 4, 4, 3, 3, 1, 4, 5, 5, 5, 1, 5, 5, 3, 3, 1, 5, 4,
       4, 3,

comments = []
for i in np.arange(0, len(df)):
    comment = df[i][11:]
    comments.append(comment)
    
print(comments)

In [31]:
df3 = pd.DataFrame({'star_rating': stars_numeric, 'review_comments': comments})

In [32]:
df3.head()

,review_comments,star_rating
0,"While the gameplay is fun, the game is not wor...",2
1,Its a very enjoyable game. What a paid app sho...,5
2,"Very fun game, it's worth the money. But there...",5
3,"It\'s a nice short game, worth the price ""free...",4
4,Its a great app but the only issue is when sol...,3


Separating into positive and negative comments

In [34]:
pos = df3[df3['star_rating'] >3]

In [35]:
neg = df3[df3['star_rating'] <=3]

In [36]:
print(len(pos), len(neg))

208 104


In [37]:
print(len(pos) + len(neg))

312


In [38]:
pos.head()

,review_comments,star_rating
1,Its a very enjoyable game. What a paid app sho...,5
2,"Very fun game, it's worth the money. But there...",5
3,"It\'s a nice short game, worth the price ""free...",4
5,Cool '),5
7,Awesome Game. I love the interface and design...,5


In [39]:
neg.head()

,review_comments,star_rating
0,"While the gameplay is fun, the game is not wor...",2
4,Its a great app but the only issue is when sol...,3
6,It does nothing Let's Just say IF I had to sh...,1
11,Fortunately I got this game free during the sa...,3
15,Didn't quite enjoy. Could've added a tutorial ...,3


In [40]:
pos.to_csv('pos_31_12_17.csv', encoding = 'utf-8')

In [41]:
neg.to_csv('neg_31_12_17.csv', encoding = 'utf-8')

In [42]:
pos_comments = pos['review_comments']

In [43]:
pos_comments

1      Its a very enjoyable game. What a paid app sho...
2      Very fun game, it's worth the money. But there...
3      It\'s a nice short game, worth the price "free...
5                                               Cool  ')
7      Awesome Game.  I love the interface and design...
8      Wow, rarely give all 5 stars! I needed this ga...
9      I dislike the app's icon.. Pls change it P.S. ...
10     I think there is a bug . When I click on scree...
12                      Good  gameplay  graphics bor  ')
13     Great game! Challenging, and logical. Finished...
14     Fun and tough. A puzzle game that's about quic...
16     Clever game. I don't feel the levels become in...
18     Really good game everything you do relates som...
19     Wonderful little gem, with an important messag...
20     I love this game.This game is great but i need...
21                         best horror games ever...  ')
24     It's pretty amazing how pixelated style Zombie...
25     Nice game, but it requir

In [136]:
def clean_text(text):
    bytes(text, 'utf-8').decode("unicode_escape")
    return text

In [138]:
clean_comments_pos = pos_comments.apply(clean_text)

In [140]:
clean_comments_pos.head()

0    I have been playing roblox since 2008 and the ...
1    I love it  But I got a different phone and log...
3    Oh my god... This is the best geometry dash fr...
5                                       I liked it  ')
7    I love it quite a lot but there are too many a...
Name: review_comments, dtype: object

In [141]:
neg_comments = neg['review_comments']

In [142]:
clean_comments_neg = neg_comments.apply(clean_text)

In [145]:
neg_comments.head()

2    It's simple, cute, and plays pretty smoothly. ...
4    It is a pretty good game, the controls r fine,...
6    Ì¢À_å_LOVE ITÌ¢À_å_just 2 humble request 1. ...
8    TOO MANY ADS  I would play this game so much m...
9    I love this game and have got really far on my...
Name: review_comments, dtype: object

<b> Starting Sentiment Analysis

In [146]:
def create_word_features(words):
    useful_words = [word for word in words if word not in stopwords.words('english')]
    my_dict = dict([(word, True) for word in useful_words])
    return my_dict

<b> Negative

In [155]:
neg_tokens=[]
for string in neg_comments:
    word = word_tokenize(string)
    neg_tokens.append((create_word_features(word), 'negative'))
    
#print(neg_tokens)
print(len(neg_tokens))    

158


<b> Positive

In [152]:
pos_tokens=[]
for string in pos_comments:
    word = word_tokenize(string)
    pos_tokens.append((create_word_features(word), 'positive'))
    
#print(pos_tokens)
print(len(pos_tokens))

[({'I': True, 'playing': True, 'roblox': True, 'since': True, '2008': True, 'way': True, 'developed': True, 'amazing': True, 'love': True, 'game': True, 'much': True, 'keep': True, 'one': True, 'issue': True, 'please': True, 'bring': True, 'tix': True, 'back': True, 'miss': True, 'know': True, 'hard': True, 'better': True, 'robux': True, 'let': True, "'s": True, 'honest': True, 'could': True, 'day': True, 'would': True, 'happy': True, 'talk': True, 'away': True, '?': True, 'Tix': True, 'made': True, ',': True, '.': True, 'Please': True, 'begging': True, '!': True, '``': True, ')': True}, 'positive'), ({'I': True, 'love': True, 'But': True, 'got': True, 'different': True, 'phone': True, 'logged': True, 'back': True, 'onto': True, 'Facebook': True, 'lost': True, 'characters': True, 'money': True, '!': True, 'really': True, 'upset': True, ',': True, 'way': True, 'get': True, 'gone': True, 'forever': True, '?': True, 'Please': True, 'help': True, 'game': True, 'would': True, 'like': True, 

<b> Create Training Set

In [156]:
train_set = pos_tokens[:162] + neg_tokens[:126]
test_set = pos_tokens[162:] + neg_tokens[126:]
print(len(train_set), len(test_set))

288 72


<b> Create the Naive Bayes Classifier

In [157]:
classifier = NaiveBayesClassifier.train(train_set)

In [160]:
accuracy = nltk.classify.util.accuracy(classifier, test_set)
print(accuracy*100)

69.44444444444444


In [166]:
fidget_review1 = """It was good I liked how I got a brand new one that fast"""
print(fidget_review) #This comment was a 5* review

It was good I liked how I got a brand new one that fast


In [167]:
words = word_tokenize(fidget_review1)
words = create_word_features(words)
classifier.classify(words)

'positive'

In [177]:
fidget_review2 = """I think that you'll did a really good game and you'll should make more games"""
print(fidget_review2) #This comment was a 5* review

I think that you'll did a really good game and you'll should make more games


In [178]:
words = word_tokenize(fidget_review2)
words = create_word_features(words)
classifier.classify(words)

'positive'

In [181]:
fidget_review3 = """Downloaded this for free money on a different game, it sucks so much. And i didnt even get the copleted ad reward, dont download uninstalling"""
print(fidget_review2) #This comment was a 1* review

I think that you'll did a really good game and you'll should make more games


In [182]:
words = word_tokenize(fidget_review3)
words = create_word_features(words)
classifier.classify(words)

'negative'